In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

# -----------------------------
# Load base + adapter weights
# -----------------------------
BASE_MODEL = "google/mt5-base"
ADAPTER_PATH = "results/adapter"
TOKENIZER_PATH = "results/tokenizer"

print("Loading tokenizer and model...")
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)
base_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL, torch_dtype=torch.float32)
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()

if torch.cuda.is_available():
    model = model.to("cuda")
    print("Model moved to GPU.")

# -----------------------------
# Inference function
# -----------------------------
def translate_singlish(sentence, max_length=128):
    prompt = f"Translate English to English:\nEnglish: {sentence}\nEnglish:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=5,
            early_stopping=True,
        )

    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

# -----------------------------
# Example runs
# -----------------------------
examples = [
    "wah the weather damn hot today leh",
    "you eat already or not?",
    "don’t play play ah, this one very expensive!",
]

for ex in examples:
    print(f"\n🗣️ Singlish: {ex}")
    print(f"💬 English: {translate_singlish(ex)}")


Loading tokenizer and model...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



🗣️ Singlish: wah the weather damn hot today leh
💬 English: <extra_id_0> English to English:

🗣️ Singlish: you eat already or not?
💬 English: <extra_id_0> English to English:

🗣️ Singlish: don’t play play ah, this one very expensive!
💬 English: <extra_id_0> English to English:


In [9]:
model.print_trainable_parameters()

trainable params: 0 || all params: 584,711,424 || trainable%: 0.0000
